Copyright 2024 - Forusone : shins777@gmail.com

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

   https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

## Grounding service with Vertex AI

# Configuration
## Install python packages
* Vertex AI SDK for Python
  * https://cloud.google.com/python/docs/reference/aiplatform/latest


In [1]:
!pip install --upgrade --quiet google-cloud-aiplatform

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 51.6 MB/s eta 0:00:00


## Authentication to access to the GCP
* Use OAuth to access the GCP environment.
 * Refer to the authentication methods in GCP : https://cloud.google.com/docs/authentication?hl=ko

In [2]:
import sys
from IPython.display import Markdown, display

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user(project_id="ai-hangsik")

!gcloud config set project ai-hangsik

Updated property [core/project].


## Set the environment on GCP Project


In [3]:
# Model configuration - Project id and region.
MODEL_NAME="gemini-1.5-flash-002"
PROJECT_ID="ai-hangsik"
REGION="asia-northeast3"

### Vertex AI initialization
Configure Vertex AI and access to the foundation model.
* Vertex AI initialization : aiplatform.init(..)
  * https://cloud.google.com/python/docs/reference/aiplatform/latest#initialization

In [4]:
import vertexai

from vertexai.generative_models import (
    GenerationConfig,
    GenerativeModel,
    Tool,
)

from vertexai.preview.generative_models import grounding

vertexai.init(project=PROJECT_ID, location=REGION)
model = GenerativeModel(MODEL_NAME)

## Response without grounding services.
This response is information from LLM without grounding services to enrich contexts sent to LLM.

##Configuration Vertex AI Search

In [11]:
# data store configuration : note : this configuration could be different with model configuration(project_id, location)
data_store_id = "daou-office-manual-ds_1732093205769"

location = "global"
project_id = "ai-hangsik"

tool = Tool.from_retrieval(
    grounding.Retrieval(
        grounding.VertexAISearch(
            datastore=data_store_id,
            project=project_id,
            location=location,
        )
    )
)

prompt ="대쉬보드에 대해서 설명해주세요."

response = model.generate_content(
    prompt,
    tools=[tool],
    generation_config=GenerationConfig(
        temperature=0.0,
    ),
)

display(Markdown(response.text))


대시보드는 특정 목적을 위해 중요한 정보를 한눈에 보여주는 사용자 인터페이스입니다.  제공된 문서에 따르면, DaouOffice 클라우드 관리자 가이드에서는 대시보드가 홈 서비스 페이지에 표시되는 가젯과 위젯을 관리하는 영역으로 설명됩니다.  

**주요 기능:**

* **가젯 및 위젯 관리:** 대시보드는 다양한 가젯(예: HTML 편집기, Quick Menu, TODO+, Works app, 메일 리스트 등)과 위젯을 표시하며, 관리자는 각 가젯의 공개 여부를 설정하여 사용자가 대시보드에서 볼 수 있는 정보를 제어할 수 있습니다.
* **대시보드 운영자 지정:** 전사 대시보드를 편집할 수 있는 운영자를 지정할 수 있습니다.  운영자는 웹 서비스를 통해 직접 대시보드를 편집할 수 있으며, 어드민 접속이 필요하지 않습니다.  운영자 추가 및 삭제는 관리자 메뉴에서 가능합니다.
* **개인 대시보드 설정:** 사용자는 자신이 사용할 가젯을 개별적으로 설정할 수 있습니다.  공개된 가젯만 개인 대시보드에 표시됩니다.


**일반적인 대시보드의 특징:**

위에서 설명한 DaouOffice의 대시보드 외에도, 일반적으로 대시보드는 다음과 같은 특징을 가지고 있습니다.

* **시각적 표현:**  차트, 그래프, 표, 지도 등 다양한 시각적 요소를 사용하여 데이터를 효과적으로 전달합니다.
* **실시간 업데이트:**  실시간으로 데이터를 업데이트하여 최신 정보를 제공하는 경우가 많습니다.
* **맞춤 설정:**  사용자의 필요에 따라 대시보드의 구성 요소와 표시되는 정보를 맞춤 설정할 수 있습니다.
* **다양한 용도:**  비즈니스 분석, 프로젝트 관리, 시스템 모니터링 등 다양한 목적으로 사용됩니다.  예를 들어,  판매 현황, 재고 관리, 고객 만족도 등을 한눈에 파악하는 데 사용될 수 있습니다.


요약하자면, 대시보드는 사용자에게 중요한 정보를 효율적으로 전달하고,  의사결정을 지원하는 중요한 도구입니다.  DaouOffice의 경우,  클라우드 서비스 관리를 위한 특화된 대시보드 기능을 제공합니다.
